In [ ]:
import torch.optim as optim
import torch 
import torch.nn.functional as F
import igl 
import networkx as nx 
import numpy as np # np.linalg.eig
import scipy as sp
from meshplot import plot, subplot, interact
import meshplot as mp

# igl 
import os
root_folder = os.getcwd()

In [ ]:
# import torch 
# import numpy as np

# # igl 
# import os
# root_folder = os.getcwd()

# %run HKS.ipynb

**Hyper Parameters**

In [ ]:
mesh_dir = '\SHREC11'
NUM_MESHES = 600 
path_to_labels = os.path.join(root_folder, "SHREC11", "labels.txt")
train_size = int(.8*NUM_MESHES) # set aside train (80% )and test (20%) data 

# set values of t 
ts= [0.001, 0.01, 0.1, 1] 

In [ ]:
# convert mesh to networkx graph

def do_task(num):
    v, f = igl.read_triangle_mesh(os.path.join(root_folder, "SHREC11", "T"+str(num)+".obj"))
    # Mesh in (v,f)
    adj_mat = igl.adjacency_matrix(f) # this gets me the adj_mat for this mesh 
    # print("type", type(adj_mat)) # type <class 'scipy.sparse._csc.csc_matrix'>
    
    G = nx.from_scipy_sparse_array(adj_mat) # creates a new graph from an adj matrix given as a Scipy sparse array 
    # print("G", G)
    
    L = nx.laplacian_matrix(G).toarray() # get Laplacian matrix  
    # print("L", L)
    
    # get eigenvalues from graph Laplacian 
    eigen_values = np.linalg.eigvals(L)
    # print("eigen_values", eigen_values)
    # print("eigen_values type", type(eigen_values)) # ndarray
    # print("eigen_values len", len(eigen_values)) # 252 nodes hence len is 252 
    
    # compute e^t*eigen_value
    
    HKS = [] 
    
    for t in ts:
        t_eigen_values = t*eigen_values # an array where each lambda is multiplied by t 
        # print("t_eigen_values", t*eigen_values)

        h_t = np.mean(np.exp(t_eigen_values)) # get average to compute h(t) 
        # print("h_t", h_t)
        
        HKS.append(h_t) 
        
    return HKS 
    


In [ ]:
HKS_all = [] 

for i in range(600):
    HKS = do_task(i)
    HKS_all.append(HKS)
    
# print("HKS len", len(HKS_all))


In [ ]:
#I have a list that contains the HKS vectors for each mesh, and I will 
# convert that into a tensor. The objects numbered 0 through 599 are scrambled
# and not organised by category but there is a labels.txt file that groups them
# by category, so using that labels.txt file, I will create a list parallel to the
# HKS vectors that contains the group number for each mesh, and convert that into a tensor. 


In [ ]:
# organise labels - taken from Davidson and Richard's code 
def readLbl(size,fileName):
    #takes in file name, returns the labels as an array
    file1 = open(fileName, 'r')
    Lines = file1.readlines()

    count = 0

    lbls = np.empty([size])
    obj_order_by_grp = np.empty([size]) # list of obj name ordered by grp no.
    lbls_order_by_grp = np.empty([size])  # list of grps ordered by grp no.
    # Strips the newline character
    for line in Lines:
        count += 1
        text = line.strip()[1:].split('.')
        text[1] = text[1].split(' ')[1]
        
        # list of obj name ordered by grp no.
        obj_order_by_grp[count-1] = int(text[0])
        
        # list of grps ordered by grp no.
        lbls_order_by_grp[count-1] = int(text[1])
        
        # parallel list to HKS - ordered by object no.
        lbls[int(text[0])] = int(text[1])
        #print("Line{}: {}".format(count, )))
        
        # file1.close()
    return lbls, obj_order_by_grp, lbls_order_by_grp

In [ ]:
# fName_labels = mesh_dir + 'labels.txt'
labels_np, obj_order_by_grp, lbls_order_by_grp = readLbl(NUM_MESHES, path_to_labels)

# print("obj_order_by_grp", obj_order_by_grp)
# print("lbls_order_by_grp", lbls_order_by_grp)

In [ ]:
# obtain labels of group 0 and 1 from larger np.arr, put them in labels_binary
# => get a np.arr that is [0,0..0,1,1..1]
labels_binary = lbls_order_by_grp[:40] # type: ndarray 

print("labels_binary", type(labels_binary))
print("labels_binary", labels_binary)

In [ ]:
# get a np.arr of meshes whose groups are 0 and 1
# this is parallel to [0,0..0,1,1..1]
obj_order_by_grp_binary = obj_order_by_grp[:40]

# convert all floats in np array to int 
a = obj_order_by_grp_binary.astype(int)

# obtain HKS belonging to group 0 and 1
HKS_binary_ls = [] 
for obj in a.tolist(): # tolist() converts type from np.arr to python list 
    el = HKS_all[obj]
    HKS_binary_ls.append(el)

# convert HKS_binary_ls back to np.array 
HKS_binary = np.array(HKS_binary_ls)

In [ ]:
# print("pre shuffle:", labels_binary)
# print("pre shuffle:", HKS_binary)

# shuffle two np arrays together
rand_indexes = np.arange(len(labels_binary))
np.random.shuffle(rand_indexes)
labels_binary=labels_binary[rand_indexes]
HKS_binary=HKS_binary[rand_indexes]

# print("post shuffle:", labels_binary)
# print("post shuffle:", HKS_binary)

In [ ]:
# training size is hardcoded bc small dataset 
labels_binary_train = torch.tensor(labels_binary[:int(.8*40)]).float()
torch.reshape(labels_binary_train, (32,1))
labels_binary_test = torch.tensor(labels_binary[int(.8*40):]).float()

HKS_binary_train = torch.tensor(HKS_binary[:int(.8*40)]).float()
HKS_binary_test = torch.tensor(HKS_binary[int(.8*40):]).float()

In [ ]:
# binary classifier 
hks_binary_classifier = torch.nn.Sequential( 
    torch.nn.Linear(4, 6), # takes in vector of size 8 (mine is 4), 16 means weight layer 
    torch.nn.Sigmoid(), # activation function, can use sigmoid etc 
    torch.nn.Linear(6, 1), #(x,y) -> y is the size of my output I'm doing a linear classifier (binary now, later 25 categories)
    torch.nn.Sigmoid()
)

In [ ]:
labels_binary_train = labels_binary_train.unsqueeze(1)

In [ ]:
# training loop 
optimizer = optim.Adam(hks_binary_classifier.parameters(), lr = 0.0001)
for i in range(2000):
    optimizer.zero_grad()
    # print(HKS_binary_train)
    output = hks_binary_classifier.forward(HKS_binary_train)
    # print(output) 
    loss = F.binary_cross_entropy(output,labels_binary_train)
    loss.backward()
    print(loss)
    optimizer.step()

In [ ]:
# multi-class classifier 
hks_binary_classifier = torch.nn.Sequential( 
    torch.nn.Linear(4, 6), # takes in vector of size 8 (mine is 4), 16 means weight layer 
    torch.nn.Tanh(), # activation function, can use sigmoid etc 
    torch.nn.Linear(6, 1), #(x,y) -> y is the size of my output I'm doing a linear classifier (binary now, later 25 categories)
    torch.nn.Softmax()
)

In [ ]:
# cast to tensor for all 600 groups
# note: everything is scrambled (group wise)
# arranged in order from T0-T599 
labels_train = torch.tensor(labels_np[:train_size]).float()
labels_test = torch.tensor(labels_np[train_size:]).float()

HKS_all_train = torch.tensor(HKS_all[:train_size]).float()
HKS_all_test = torch.tensor(HKS_all[train_size:]).float()

In [ ]:
#davidson and richard
optimizer = optim.Adam(g.weights + n.weights,lr = 0.0001)
for i in range(100000):
    optimizer.zero_grad()
    output = n.forward(g.forward(normed_adjMats_tr,node_sigs_tr))
    loss = lossF(output,label_mat_tr)
    loss.backward()
    optimizer.step()

In [ ]:
# simple network 
hks_classifier = torch.nn.Sequential( 
    torch.nn.Linear(4, 6), # takes in vector of size 8 (mine is 4), 16 means weight layer 
    torch.nn.Tanh(), # activation function, can use sigmoid etc 
    torch.nn.Linear(6, ), #(x,y) -> y is the size of my output I'm doing a linear classifier (binary now, later 25 categories)
    torch.nn.Tanh(),
)

# how to apply
# one hot encoding?? 
# this is probably what I want 
# output size is num. of categories (beginning 2, later 600 then 2400) 

# loss function:
# how far away your predictions are from actual labels
# with 1 hot, subtract predicted from actual value, take absolute value (diff) 
# take sum of the differences

